In [2]:
import requests
import os
import json
import config
import preprocessor as p #cleaner
from langdetect import detect #language detecter
from csv import writer #write to a csv file
import time



from ernie import SentenceClassifier
import numpy as np


classifer = SentenceClassifier(model_path='./modelSem')


output = {}

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers = headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(headers,delete,bearer_token):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "#bitcoin", "tag": "bitcoin"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(headers, set, bearer_token):
    t_end = time.time() + 15
    print("time set")
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", headers=headers, stream=True,  
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if time.time() > t_end:
            print("timeout")
            break
        if response_line:
            json_response = json.loads(response_line)
            #print(json.dumps(json_response, indent=4, sort_keys=True))
            tweet = json_response['data']['text']
            tweet = p.clean(tweet)
            tweet = tweet.replace(":","")
            try:
                if detect(tweet) == 'en':
                    print(tweet)
                    try: #some tweets dont have text // not included
                        classes = ['Bearish', 'Neutral', 'Bullish']
                        probabilities = classifer.predict_one(tweet)
                        print(classes[np.argmax(probabilities)])
                        output[tweet] = classes[np.argmax(probabilities)]
                    except:
                        pass
            except:
                pass
    print("before break")
    return output

def main():
    bearer_token = config.BEARER_TOKEN
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    output = get_stream(headers, set, bearer_token)
    print("ended")
    
    

if __name__ == "__main__":
    main()

{"data": [{"id": "1520964468020330496", "value": "#bitcoin", "tag": "bitcoin"}], "meta": {"sent": "2022-05-02T03:12:50.729Z", "result_count": 1}}
{"meta": {"sent": "2022-05-02T03:12:51.636Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "#bitcoin", "tag": "bitcoin", "id": "1520964509233557504"}], "meta": {"sent": "2022-05-02T03:12:52.997Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
time set
200
You mean the guy with the flip phone?
Bullish
 is universal money. Central African Republic president
Bullish
 Between helping and I think these guys have their hearts in right place. Here's Trump endorsing
Neutral
 / US Debt by years until maturity, a thread.
Bullish
May st, Market Market Statistics - Total market capitalization of $1,782 trillion. - -hour trading volume $93 billion. - Liquidated traders with total assets of $373.19 million (over % long positions). - Fear and Greed Index Extreme
Bullish
Meme season coming, just found this GEM .

In [3]:

#DATA SET CREATOR CODE


def get_stream(headers, set, bearer_token):
    t_end = time.time() + 5
    print("time set")
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", headers=headers, stream=True,  
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            tweet = json_response['data']['text']
            tweet = p.clean(tweet)
            try:
                if detect(tweet) == 'en':
                    print(tweet)
                    tweetlst = [tweet]
                    with open('dataset.csv', 'a+', newline='') as write_obj:
                    csv_writer = writer(write_obj)
                    csv_writer = writerow(tweetlst)
            except:
                pass

IndentationError: expected an indented block (2826133897.py, line 27)